<a href="https://colab.research.google.com/github/FredPedrosa/backtranslationllm/blob/main/BackTranslationLLM_CISMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BackTranslationLLM: Validade de Conteúdo com IA Agêntica

Este notebook implementa uma pipeline automatizada para acelerar o processo psicométrico de Validade de Conteúdo, utilizando uma abordagem de múltiplos agentes de Inteligência Artificial. A metodologia combina a técnica de tradução reversa (*back-translation*) com um comitê de juízes virtuais para avaliar a qualidade dos itens de um instrumento.

Para utilizar esta ferramenta, é **essencial** possuir as seguintes chaves de API:
1.  **Google AI:** Crie uma chave de API no [Google AI Studio](https://aistudio.google.com/app/apikey). A cota gratuita é generosa e o modelo é *pay-as-you-go* (pague pelo que usar) após o término.
2.  **DeepL:** Crie uma conta para obter uma chave de API (o plano gratuito é suficiente).

---
## 1. Arquitetura Agêntica e Modelos Selecionados

A pipeline é dividida em duas fases principais, cada uma utilizando modelos específicos para garantir alta performance, confiabilidade e diversidade de análise.

### Fase 1: Pipeline de Tradução e Refinamento

Nesta fase, os agentes trabalham em sequência para produzir uma tradução de alta fidelidade, garantindo equivalência semântica e aderência às regras metodológicas do instrumento.

| Função do Agente | Modelo Selecionado | Justificativa da Escolha |
| :--- | :--- | :--- |
| **Agente Tradutor**<br>*(Tradução Inicial)* | **DeepL API** | Mantido por ser uma API especializada e de altíssima qualidade para a tarefa de tradução, servindo como uma excelente base para o processo. |
| **Agente Verificador**<br>*(Retrotradução)* | **Gemini 1.5 Flash** | É o modelo mais rápido e com melhor custo-benefício da família Gemini. A retrotradução é uma tarefa mais simples que não exige o máximo de raciocínio, apenas uma tradução literal de volta ao idioma original para verificação semântica. O `Flash` é perfeito para isso. |
| **Agente Juiz**<br>*(Auditoria de Qualidade)* | **Gemini 1.5 Pro** | É o "cérebro" da operação. Esta tarefa exige raciocínio complexo para interpretar as regras do `CONTEXTO` e gerar uma saída estruturada em JSON. O `Pro` é o modelo mais indicado para tarefas que exigem alta capacidade de seguir instruções e lógica. |
| **Agente Refinador**<br>*(Correção de Erros)* | **Gemini 1.5 Pro** | Similar ao Juiz, o Refinador precisa de uma forte capacidade de compreensão de contexto. Ele deve entender o "motivo do erro" e gerar uma nova versão corrigida. O `Pro` garante a melhor performance para esta tarefa de geração de texto refinado. |

### Fase 2: Comitê de Juízes para Validade de Conteúdo (CVC)

Após a tradução, um painel diversificado de LLMs atua como juízes especialistas. Para simular um comitê multidisciplinar, utilizamos diferentes modelos das famílias Gemini e Gemma, cada um com uma "persona" específica.

| Persona do Juiz | Modelo Selecionado | Justificativa da Escolha |
| :--- | :--- | :--- |
| **Juiz 1**<br>*(Psicometrista)* | **Gemini Flash (ex: `gemini-1.5-flash`)** | Um modelo rápido e moderno, ideal para a persona do psicometrista que precisa de avaliações objetivas e eficientes, focando na clareza e estrutura sem o custo do modelo `Pro`. |
| **Juiz 2**<br>*(Linguista)* | **Gemini Pro (ex: `gemini-1.5-pro`)** | Atribuímos o modelo mais poderoso à análise linguística, que exige uma compreensão profunda de nuances gramaticais, fluidez e naturalidade da linguagem, tarefas onde o `Pro` se destaca. |
| **Juiz 3**<br>*(PhD em Musicoterapia)* | **Gemma (versão grande, ex: `gemma-2-9b-it`)** | Usar um modelo da família Gemma introduz uma arquitetura de treinamento diferente. Atribuído à persona teórica, ele pode fornecer uma "opinião" com vieses distintos dos modelos Gemini, enriquecendo a avaliação do construto. |
| **Juiz 4**<br>*(Tradutor Cultural)* | **Gemini Pro Preview** | Utilizar uma versão "preview" ou experimental do Gemini `Pro` simula um especialista que está na "vanguarda", potencialmente com acesso a dados mais recentes. É uma ótima escolha para a persona que avalia vieses e regionalismos. |
| **Juiz 5**<br>*(Musicoterapeuta Clínico)* | **Gemma (versão pequena, ex: `gemma-7b-it`)** | Um modelo Gemma menor simula um especialista com foco na praticidade e aplicação direta. Sua avaliação pode ser mais pragmática, e sua arquitetura diferente complementa o painel, garantindo máxima diversidade de "perspectivas". |

In [ ]:
#@title **Rode para instalação e configuração do ambiente**

# Instala as dependências de forma segura
!pip install deepl google-generativeai ipywidgets -q > /dev/null

# --- Importações Globais ---
import json
import time
import deepl
import google.generativeai as genai
from google.generativeai.types import GenerationConfig
from IPython.display import display, HTML

# Carrega as chaves de API secretas
from google.colab import userdata
DEEPL_API_KEY = userdata.get('DEEPL_API_KEY')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY2')

print("✅ Dependências instaladas e chaves de API carregadas.")

✅ Dependências instaladas e chaves de API carregadas.


In [ ]:
#@title **2. Idiomas, Contexto e Temperatura (ES -> PT-BR)**:

# --- 2.1 - Configuração de Idiomas ---
IDIOMA_ORIGINAL = "Espanhol"
IDIOMA_ALVO = "Português Brasileiro"

# --- 2.2 - Contexto e Manual de Estilo para os Juízes (em Português) ---
CONTEXTO = """
**Objetivo:** Este instrumento, chamado CISMA, mede o impacto de sessões de musicoterapia no bem-estar de adultos. A tradução deve ser adequada para o português brasileiro.

**--- Manual de Estilo e Diretrizes Metodológicas (Regras) ---**

1.  **Equivalência Semântica:** A tradução deve manter o significado exato do item original em espanhol. Deve-se preservar a temporalidade imediata dos itens (foco no "aqui e agora"), evitando interpretações que generalizem o estado do paciente.

2.  **Estrutura do Item (CRÍTICO):** As perguntas devem ser interrogativas diretas (ex: 'Eu sinto... ?', 'A musicoterapia me ajudou...?').

3.  **Perspectiva Narrativa (CRÍTICO):** Este é um instrumento de autorrelato. Os itens devem ser formulados na perspectiva da primeira pessoa ('Eu', 'meu', 'me'). A tradução NÃO PODE mudar para a segunda pessoa ('você', 'seu'). Esta regra é crítica.

4.  **Clareza e Simplicidade:** A linguagem deve ser simples, direta e sem ambiguidades, adequada para pacientes. Cada item deve fazer apenas UMA pergunta. As traduções devem ser concisas, mantendo o espírito de um instrumento "eficiente e breve", sem jargões técnicos.

5.  **Tom:** O tom deve ser neutro e profissional, adequado para um instrumento formal de autorrelato. Evitar gírias ou linguagem excessivamente casual.

6.  **Padrão de Avaliação:** Sua avaliação deve ser extremamente rigorosa. Mesmo desvios menores dessas diretrizes, especialmente em relação à **Estrutura do Item (Regra #2)** e à **Perspectiva Narrativa (Regra #3)**, são considerados falhas significativas.
"""

# --- 2.3 - Configuração do Modelo de IA ---
TEMPERATURA = 0.2

print("✅ Parâmetros e Manual de Estilo (versão final) configurados para a tradução de Espanhol para Português Brasileiro.")

✅ Parâmetros e Manual de Estilo (versão final) configurados para a tradução de Espanhol para Português Brasileiro.


In [ ]:
#@title **Upload de arquivo .csv e leitura robusta**

from google.colab import userdata, files
import pandas as pd
import io

# 1.2 - FUNÇÃO FINAL E MAIS ROBUSTA PARA LER ITENS
def ler_itens_do_arquivo_de_texto(file_content):
    """Lê um arquivo linha por linha, tratando-o como texto simples. A prova de falhas para pontuação."""
    potential_encodings = ['utf-8-sig', 'utf-8', 'latin-1', 'cp1252']

    for encoding in potential_encodings:
        try:
            # Decodifica todo o conteúdo do arquivo para uma única string de texto
            text_data = file_content.decode(encoding)

            # Divide a string em uma lista de linhas
            lines = text_data.splitlines()

            # Limpa a lista: remove espaços em branco extras e linhas vazias
            items = [line.strip() for line in lines if line.strip()]

            # Se encontramos itens, a leitura foi um sucesso
            if items:
                print(f"✅ Arquivo lido com sucesso como texto simples (encoding='{encoding}')")
                return items
        except Exception:
            # Se a decodificação falhar, tenta o próximo encoding
            continue

    # Se nenhum encoding funcionar
    return None

# 1.3 - Upload do Arquivo
print("\n--- ETAPA 1: UPLOAD DO ARQUIVO ---")
print("Por favor, faça o upload do seu arquivo CSV/TXT com os itens:")
uploaded = files.upload()

NOME_DO_ARQUIVO_CSV = list(uploaded.keys())[0]
conteudo_arquivo = uploaded[NOME_DO_ARQUIVO_CSV]
itens_lidos = ler_itens_do_arquivo_de_texto(conteudo_arquivo)

if itens_lidos:
    print(f"\n--- Pré-visualização dos {len(itens_lidos)} Itens de '{NOME_DO_ARQUIVO_CSV}' ---")
    for item in itens_lidos[:5]: # Mostra os 5 primeiros itens para verificação
        print(f"- {item}")
    print("--------------------------------------------------")
else:
    print("\n⚠️ Não foi possível extrair itens do arquivo. Verifique o conteúdo do arquivo.")

print("\n✅ Arquivo pronto! Agora configure os parâmetros na Célula 2.")


--- ETAPA 1: UPLOAD DO ARQUIVO ---
Por favor, faça o upload do seu arquivo CSV/TXT com os itens:


Saving CISMA.csv to CISMA (1).csv
✅ Arquivo lido com sucesso como texto simples (encoding='utf-8-sig')

--- Pré-visualização dos 14 Itens de 'CISMA (1).csv' ---
- ¿Cómo me encuentro ahora?
- ¿Cuánto dolor tengo en este momento?
- ¿Me siento relajado/a?
- ¿Me siento animado/a?
- ¿Me siento acompañado/a?
--------------------------------------------------

✅ Arquivo pronto! Agora configure os parâmetros na Célula 2.


In [ ]:
#@title **Comitê de tradução reversa**

#======================================================================
# INICIALIZAÇÃO E ARQUITETURA
#======================================================================
print("⚙️ Inicializando clientes de API...")
try:
    generation_config = GenerationConfig(temperature=TEMPERATURA)
    genai.configure(api_key=GEMINI_API_KEY)
    deepl_translator = deepl.Translator(DEEPL_API_KEY)
    gemini_pro_model = genai.GenerativeModel('models/gemini-pro-latest')
    gemini_flash_model = genai.GenerativeModel('models/gemini-flash-latest')
    print("✅ Clientes de API prontos.")
except Exception as e:
    raise RuntimeError(f"Falha ao inicializar: {e}")

# --- FUNÇÃO DE RETENTATIVA ROBUSTA ---
def call_gemini_with_retry(agent_name, model, prompt, config, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt, generation_config=config)
            return response
        except Exception as e:
            error_message = str(e).lower()
            if any(code in error_message for code in ['429', '500', '503', 'resource exhausted']):
                wait_time = 20 * (2 ** attempt)
                print(f"  [AVISO {agent_name}] Erro temporário detectado. Esperando {wait_time}s... ({attempt + 1}/{max_retries})")
                time.sleep(wait_time)
            else:
                print(f"  [ERRO GERAL {agent_name}] Erro não recuperável: {e}. Falha imediata.")
                return None
    print(f"  [ERRO FINAL {agent_name}] Todas as {max_retries} tentativas falharam."); return None

# (Os agentes e o resto do motor permanecem os mesmos)
class TranslatorAgent:
    def execute(self, text, target_lang_code):
        try: return deepl_translator.translate_text(text, target_lang=target_lang_code).text
        except Exception as e: print(f"  [ERRO AgenteTradutor] {e}"); return "ERRO: DeepL falhou."
class VerifierAgent:
    def execute(self, text, source_lang, context):
        prompt = f"Context: {context}\nTranslate the following English sentence to {source_lang}.\nProvide ONLY the translated sentence.\nSentence: '{text}'"
        response = call_gemini_with_retry("AgenteVerificador", gemini_flash_model, prompt, generation_config)
        return response.text.strip().replace('*', '') if response else "ERRO: Gemini Flash falhou."
class JudgeAgent:
    def execute(self, original, back_translated, initial_translation, context):
        prompt = f"""
        Your role is a Quality Auditor. You must evaluate the 'Initial Translation' based on two criteria, using the provided 'Context' as your official rulebook.
        1. **Semantic Equivalence**: First, compare the 'Original' and 'Back-Translated' sentences. Are they semantically equivalent?
        2. **Guideline Adherence**: Second, carefully read the 'Context'. It contains specific guidelines and rules (e.g., preferred terminology). Does the 'Initial Translation' strictly adhere to ALL guidelines mentioned in the 'Context'?
        Context / Rulebook: "{context}"
        Original: "{original}"
        Back-Translated: "{back_translated}"
        Initial Translation: "{initial_translation}"
        Respond ONLY with a valid JSON object with two keys:
        {{"is_approved": <true if BOTH criteria are met, otherwise false>, "reasoning": "<A concise explanation for your final decision, referencing the context if a rule was broken>"}}
        """
        response = call_gemini_with_retry("AgenteJuiz", gemini_pro_model, prompt, generation_config)
        if response:
            response_text = response.text
            json_start = response_text.find('{'); json_end = response_text.rfind('}') + 1
            if json_start != -1 and json_end != -1: return json.loads(response_text[json_start:json_end])
            else: return {"is_approved": False, "reasoning": "Interpretação: A resposta do modelo não foi um JSON válido."}
        return {"is_approved": None, "reasoning": "ERRO: Gemini Pro falhou após retentativas."}
class RefinerAgent:
    def execute(self, original, problematic_translation, reasoning, context, target_lang):
        prompt = f"You are an expert translator. Your task is to refine a translation for a questionnaire about '{context}'.\nOriginal sentence: \"{original}\"\nProblematic translation: \"{problematic_translation}\"\nReasoning for the problem: \"{reasoning}\"\nProvide an improved translation to {target_lang} that fixes the issue. Return ONLY the refined sentence."
        response = call_gemini_with_retry("AgenteRefinador", gemini_pro_model, prompt, generation_config)
        return response.text.strip() if response else "ERRO: Gemini Pro falhou."
def run_orchestrated_pipeline(items, config):
    translator, verifier, judge, refiner = TranslatorAgent(), VerifierAgent(), JudgeAgent(), RefinerAgent()
    full_report = []; print(f"\n🚀 Orquestrador iniciando pipeline para {len(items)} itens...")
    for i, item_original in enumerate(items):
        print(f"\n--- Processando Item {i+1}/{len(items)}: '{item_original}' ---")
        report_entry = {"item_original": item_original}
        translation_v1 = translator.execute(item_original, config['deepl_code']); report_entry["etapa_1_traducao_inicial"] = translation_v1
        print(f"  1. Tradução (DeepL): '{translation_v1}'")
        if "ERRO:" in str(translation_v1): full_report.append(report_entry); continue
        back_translation = verifier.execute(translation_v1, config['source_lang'], config['context']); report_entry["etapa_2_retrotraducao"] = back_translation
        print(f"  2. Retrotradução (Gemini Flash): '{back_translation}'")
        if "ERRO:" in str(back_translation): full_report.append(report_entry); continue
        judgement = judge.execute(item_original, back_translation, translation_v1, config['context']); report_entry["etapa_3_julgamento"] = judgement
        print(f"  3. Julgamento (Gemini Pro): Aprovado? {judgement.get('is_approved')}")
        if judgement.get("is_approved"):
            report_entry["status_final"] = "APROVADO"; report_entry["traducao_final"] = translation_v1
        else:
            reasoning = judgement.get("reasoning", "N/A")
            refined_translation = refiner.execute(item_original, translation_v1, reasoning, config['context'], config['target_lang'])
            report_entry["etapa_4_traducao_corrigida"] = refined_translation; report_entry["status_final"] = "CORRIGIDO"; report_entry["traducao_final"] = refined_translation
            print(f"  4. Correção (Gemini Pro): '{refined_translation}'")
        full_report.append(report_entry)
        time.sleep(2)
    print("\n✅ Orquestrador concluiu!"); return full_report
def generate_html_report(report_data, config):
    html = f"""
    <html><head><title>Relatório de Tradução</title><style>body {{ font-family: sans-serif; margin: 2em; }} .report-container {{ max-width: 900px; margin: auto; }} .item-card {{ border: 1px solid #ddd; border-radius: 8px; margin-bottom: 2em; padding: 1.5em; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }} .status-aprovado {{ border-left: 5px solid #28a745; }} .status-corrigido {{ border-left: 5px solid #ffc107; }} .status-falha {{ border-left: 5px solid #dc3545; }} h1, h2, h3 {{ color: #333; }} h3 {{ border-bottom: 1px solid #eee; padding-bottom: 0.5em; }} table {{ width: 100%; border-collapse: collapse; }} th, td {{ text-align: left; padding: 8px; vertical-align: top; }} th {{ width: 25%; }} tr {{ border-bottom: 1px solid #f0f0f0; }} code {{ background-color: #f5f5f5; padding: 2px 5px; border-radius: 4px; }}</style></head><body><div class="report-container"><h1>Relatório do Processo de Tradução</h1><p><strong>De:</strong> {config['source_lang']}</p><p><strong>Para:</strong> {config['target_lang']}</p><p><strong>Contexto:</strong> {config['context']}</p><hr>"""
    for i, item in enumerate(report_data):
        status = item.get('status_final', 'FALHA'); status_class = f"status-{status.lower()}"
        html += f"""<div class="item-card {status_class}"><h2>Item {i+1}: {status}</h2><table><tr><th>Original ({config['source_lang']})</th><td><code>{item.get('item_original', '')}</code></td></tr><tr><th>Tradução Inicial (DeepL)</th><td><code>{item.get('etapa_1_traducao_inicial', '')}</code></td></tr><tr><th>Retrotradução (Gemini Flash)</th><td><code>{item.get('etapa_2_retrotraducao', '')}</code></td></tr><tr><th>Julgamento (Gemini Pro)</th><td><strong>Aprovado:</strong> {item.get('etapa_3_julgamento', {}).get('is_approved', 'N/A')}<br><strong>Raciocínio:</strong> {item.get('etapa_3_julgamento', {}).get('reasoning', 'N/A')}</td></tr>"""
        if status == 'CORRIGIDO': html += f"""<tr><th>Tradução Corrigida (Gemini Pro)</th><td><code>{item.get('etapa_4_traducao_corrigida', '')}</code></td></tr>"""
        html += f"""<tr><th><strong>Tradução Final</strong></th><td><strong><code>{item.get('traducao_final', 'N/A')}</code></strong></td></tr></table></div>"""
    html += "</div></body></html>"; return html

#======================================================================
# EXECUÇÃO PRINCIPAL
#======================================================================
LANG_MAP = {"Português Brasileiro": "PT-BR", "Inglês Americano": "EN-US", "Inglês Britânico": "EN-GB", "Espanhol": "ES", "Francês": "FR", "Alemão": "DE"}

try:
    if 'itens_lidos' not in globals() or not itens_lidos:
        raise NameError("Os itens não foram lidos. Por favor, execute a Célula 2.")

    CONFIG = {"source_lang": IDIOMA_ORIGINAL, "target_lang": IDIOMA_ALVO, "deepl_code": LANG_MAP[IDIOMA_ALVO], "context": CONTEXTO.strip()}

    # --- MUDANÇA CRUCIAL: 'report_data' agora é global ---
    report_data = run_orchestrated_pipeline(itens_lidos, CONFIG)

    report_html = generate_html_report(report_data, CONFIG)
    html_filename = "relatorio_traducao_completo.html"
    json_filename = "relatorio_traducao_dados.json"
    with open(html_filename, "w", encoding="utf-8") as f: f.write(report_html)
    with open(json_filename, "w", encoding="utf-8") as f: json.dump(report_data, f, ensure_ascii=False, indent=2)

    print("\n✅ Relatórios gerados! Fazendo download...")
    files.download(html_filename)
    files.download(json_filename)

    print("\n--- Visualização do Relatório Final ---")
    display(HTML(report_html))

    print("\n\n✅✅✅ Processo de tradução concluído. A variável 'report_data' está pronta.")
    print("‼️ Agora, execute a Célula 4 para iniciar a análise dos juízes e o cálculo do CVC.")

except Exception as e:
    print(f"🚨 UM ERRO INESPERADO OCORREU: {e}")

#try:
    # --- [MUDANÇA CRUCIAL] ---
    # Verifica se a variável 'itens_lidos' foi criada pela Célula 3.
#    if 'itens_lidos' not in globals() or not itens_lidos:
#        raise NameError("Os itens não foram lidos. Por favor, execute a Célula 3 (Upload de arquivo) antes de rodar esta.")

    # [REMOVIDO] Não há mais leitura de arquivo aqui. Apenas usamos o que já foi lido.

#    print(f"✅ Itens extraídos da Célula 3 prontos para o pipeline. Total: {len(itens_lidos)}.")

#    CONFIG = {"source_lang": IDIOMA_ORIGINAL, "target_lang": IDIOMA_ALVO, "deepl_code": LANG_MAP[IDIOMA_ALVO], "context": CONTEXTO.strip()}

    # A função agora usa 'itens_lidos' diretamente.
#    report_data = run_orchestrated_pipeline(itens_lidos, CONFIG)

#    report_html = generate_html_report(report_data, CONFIG)
#    html_filename = "relatorio_traducao_completo.html"; json_filename = "relatorio_traducao_dados.json"
#    with open(html_filename, "w", encoding="utf-8") as f: f.write(report_html)
#    with open(json_filename, "w", encoding="utf-8") as f: json.dump(report_data, f, ensure_ascii=False, indent=2)
#    print("\n✅ Relatórios gerados! Fazendo download..."); files.download(html_filename); files.download(json_filename)
#    print("\n--- Visualização do Relatório Final ---"); display(HTML(report_html))
#except Exception as e:
#    print(f"🚨 UM ERRO INESPERADO OCORREU: {e}")

⚙️ Inicializando clientes de API...
✅ Clientes de API prontos.

🚀 Orquestrador iniciando pipeline para 14 itens...

--- Processando Item 1/14: '¿Cómo me encuentro ahora?' ---
  1. Tradução (DeepL): 'Como estou me sentindo agora?'
  2. Retrotradução (Gemini Flash): '¿Cómo me siento ahora?'
  3. Julgamento (Gemini Pro): Aprovado? True

--- Processando Item 2/14: '¿Cuánto dolor tengo en este momento?' ---
  1. Tradução (DeepL): 'Quanta dor estou sentindo neste momento?'
  2. Retrotradução (Gemini Flash): '¿Cuánto dolor estoy sintiendo en este momento?'
  3. Julgamento (Gemini Pro): Aprovado? True

--- Processando Item 3/14: '¿Me siento relajado/a?' ---
  1. Tradução (DeepL): 'Eu me sinto relaxado?'
  2. Retrotradução (Gemini Flash): '¿Me siento relajado?'
  3. Julgamento (Gemini Pro): Aprovado? False
  4. Correção (Gemini Pro): 'Eu me sinto relaxado/a?'

--- Processando Item 4/14: '¿Me siento animado/a?' ---
  1. Tradução (DeepL): 'Eu me sinto encorajado?'
  2. Retrotradução (Gemini Flash

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


--- Visualização do Relatório Final ---


Original (Espanhol),¿Cómo me encuentro ahora?
Tradução Inicial (DeepL),Como estou me sentindo agora?
Retrotradução (Gemini Flash),¿Cómo me siento ahora?
Julgamento (Gemini Pro),"Aprovado: TrueRaciocínio: A tradução está em conformidade com todas as diretrizes. A equivalência semântica entre o original e a retrotradução é mantida. A tradução segue as regras críticas, utilizando uma estrutura de pergunta interrogativa direta (Regra #2) e a perspectiva narrativa em primeira pessoa (Regra #3)."
Tradução Final,Como estou me sentindo agora?
Original (Espanhol),¿Cuánto dolor tengo en este momento?
Tradução Inicial (DeepL),Quanta dor estou sentindo neste momento?
Retrotradução (Gemini Flash),¿Cuánto dolor estoy sintiendo en este momento?
Julgamento (Gemini Pro),"Aprovado: TrueRaciocínio: A tradução atende a todos os critérios. A equivalência semântica entre o original e o retrotraduzido é mantida, com foco no estado presente ('neste momento'). A tradução adere estritamente às diretrizes críticas: é uma pergunta interrogativa direta (Regra #2) e está na perspectiva da primeira pessoa ('estou sentindo') (Regra #3). A linguagem é clara, simples e apropriada para o contexto."
Tradução Final,Quanta dor estou sentindo neste momento?
Original (Espanhol),¿Me siento relajado/a?




✅✅✅ Processo de tradução concluído. A variável 'report_data' está pronta.
‼️ Agora, execute a Célula 4 para iniciar a análise dos juízes e o cálculo do CVC.


In [ ]:
#@title **Salvar Resultados da Execução Anterior (Compatível com Excel)**

import pandas as pd
from google.colab import files

print("💾 Acessando os resultados da última execução do pipeline...")

try:
    # 1. Verifica se a variável 'report_data' existe na memória.
    if 'report_data' in globals() and report_data:

        # 2. Extrai as colunas de interesse
        final_items_data = {
            'item_original': [item.get('item_original', 'N/A') for item in report_data],
            'traducao_final': [item.get('traducao_final', 'N/A') for item in report_data]
        }

        # 3. Cria um DataFrame do pandas
        df_final = pd.DataFrame(final_items_data)

        # 4. Salva em um arquivo CSV e aciona o download
        nome_arquivo_final = "traducoes_finais.csv"

        # [MUDANÇA CRUCIAL] Adicionando encoding='utf-8-sig' para máxima compatibilidade com o Excel.
        # O 'sig' (signature) adiciona um BOM (Byte Order Mark) que ajuda o Excel a identificar
        # a codificação UTF-8 corretamente, exibindo caracteres especiais (¿, á, ç) sem problemas.
        df_final.to_csv(nome_arquivo_final, index=False, sep=';', encoding='utf-8-sig')

        files.download(nome_arquivo_final)

        print(f"✅ Arquivo '{nome_arquivo_final}' salvo com sucesso e formatado para melhor compatibilidade!")
        print("--- Pré-visualização do que foi salvo ---")
        display(df_final.head())

    else:
        print("⚠️ Erro: A variável 'report_data' não foi encontrada ou está vazia. Por favor, execute a célula anterior primeiro.")

except Exception as e:
    print(f"🚨 Um erro inesperado ocorreu ao tentar salvar os dados: {e}")

💾 Acessando os resultados da última execução do pipeline...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Arquivo 'traducoes_finais.csv' salvo com sucesso e formatado para melhor compatibilidade!
--- Pré-visualização do que foi salvo ---


,item_original,traducao_final
0,¿Cómo me encuentro ahora?,Como estou me sentindo agora?
1,¿Cuánto dolor tengo en este momento?,Quanta dor estou sentindo neste momento?
2,¿Me siento relajado/a?,Eu me sinto relaxado/a?
3,¿Me siento animado/a?,Eu me sinto animado/a?
4,¿Me siento acompañado/a?,Eu me sinto acompanhado?




---



#ANTES DE SEGUIR, **REINICIE O AMBIENTE DE EXECUÇÃO**

---




In [ ]:
# Célula 4: DEFINIÇÃO DO COMITÊ DE JUÍZES (Versão Corrigida)

import google.generativeai as genai
import json

#@title **Definição do Comitê de Juízes Virtuais**
print("⚙️ Montando o comitê de juízes de IA com diferentes especialidades...")

# --- Mapa global para converter texto em número ---
CATEGORY_TO_SCORE_MAP = {
    # Clareza
    "Incomprehensible": 1, "Barely Clear": 2, "Clear": 3, "Very Clear": 4, "Perfectly Clear": 5,
    # Pertinência
    "Not Pertinent": 1, "Barely Pertinent": 2, "Pertinent": 3, "Very Pertinent": 4, "Perfectly Pertinent": 5,
    # Relevância
    "Not Relevant": 1, "Barely Relevant": 2, "Relevant": 3, "Very Relevant": 4, "Perfectly Relevant": 5,
}

# --- A classe que define o comportamento de um juiz individual ---
class ContentValidationJudgeAgent:
    # A definição da classe começa aqui, sem duplicação
    def __init__(self, persona, model):
        self.persona = persona
        self.model = model

    def evaluate(self, original_item, translated_item, context):
        prompt = f"""
        You are a meticulous Quality Auditor for psychometric instruments. Your specific persona is: '{self.persona}'.
        Your primary mission is to enforce the rules described in the 'Context / Rulebook'.

        Evaluate the 'Translated Item' with extreme rigor against the guidelines. Pay special attention to **Item Structure (Rule #2)** and **Clarity (Rule #3)** from the rulebook.

        Context / Rulebook: "{context}"

        Original Item: "{original_item}"
        Translated Item: "{translated_item}"

        First, make a categorical judgment for each criterion based on the Rulebook. Then, provide the final JSON output.

        **Criteria & Categorical Choices:**
        1.  **Clarity:** [Incomprehensible, Barely Clear, Clear, Very Clear, Perfectly Clear]
        2.  **Practical Pertinence:** [Not Pertinent, Barely Pertinent, Pertinent, Very Pertinent, Perfectly Pertinent]
        3.  **Theoretical Relevance:** [Not Relevant, Barely Relevant, Relevant, Very Relevant, Perfectly Relevant]

        Respond ONLY with a valid JSON object.
        Format: {{"clarity_category": "<Your_Choice>", "pertinence_category": "<Your_Choice>", "relevance_category": "<Your_Choice>", "brief_justification": "<A very short justification, explicitly referencing a rule from the Rulebook if it was broken>"}}
        """
        # A função de retentativa (call_gemini_with_retry) deve ter sido definida em uma célula anterior (Célula 3)
        # É importante que a Célula 3, que define essa função, seja executada antes desta.
        response = call_gemini_with_retry(f"Judge ({self.persona})", self.model, prompt, generation_config)
        if response:
            try:
                response_text = response.text
                json_start = response_text.find('{'); json_end = response_text.rfind('}') + 1
                if json_start != -1 and json_end != -1:
                    return json.loads(response_text[json_start:json_end])
            except (json.JSONDecodeError, AttributeError):
                print(f"  [WARNING Judge] Invalid model response, assigning error.")
        return {"clarity_category": "Error", "pertinence_category": "Error", "relevance_category": "Error", "brief_justification": "Error during judge evaluation."}

# --- Configuração do Comitê ---
committee_setup = [
    {
        "persona": "Psychometrician (focused on clarity and objectivity)",
        "model_name": 'models/gemini-2.5-flash'
    },
    {
        "persona": "Linguist (focused on grammar and natural phrasing)",
        "model_name": 'models/gemini-2.5-pro'
    },
    {
        "persona": "Music Therapist PhD (focused on the theoretical construct)",
        "model_name": 'models/gemma-3-27b-it'
    },
    {
        "persona": "Cultural Translator (focused on biases and regionalisms)",
        "model_name": 'models/gemini-2.5-pro-preview-06-05'
    },
    {
        "persona": "Clinical Music Therapist (focused on practical application)",
        "model_name": 'models/gemma-3-4b-it'
    }
]

# --- Montagem do Comitê ---
committee = []
for setup in committee_setup:
    model_instance = genai.GenerativeModel(setup["model_name"])
    judge_agent = ContentValidationJudgeAgent(persona=setup["persona"], model=model_instance)
    committee.append(judge_agent)

print(f"✅ Comitê com {len(committee)} juízes pronto.")
print("‼️ Agora, execute a próxima célula para carregar o arquivo com as traduções.")

⚙️ Montando o comitê de juízes de IA com diferentes especialidades...
✅ Comitê com 5 juízes pronto.
‼️ Agora, execute a próxima célula para carregar o arquivo com as traduções.


In [ ]:
# Célula 5: UPLOAD E LEITURA DOS ITENS TRADUZIDOS

from google.colab import files
import pandas as pd
import io
from IPython.display import display

#@title **Carregue o arquivo com os itens traduzidos para a análise**

print("Por favor, faça o upload do arquivo 'traducoes_finais.csv' que foi salvo pela Célula 3.")
uploaded_for_validation = files.upload()

try:
    # Pega o nome e o conteúdo do arquivo que acabou de ser carregado
    validation_filename = list(uploaded_for_validation.keys())[0]
    validation_content = uploaded_for_validation[validation_filename]

    # Lê o arquivo CSV. É crucial usar sep=';' porque foi assim que salvamos para evitar problemas com vírgulas.
    df_translations = pd.read_csv(io.BytesIO(validation_content), sep=';')

    # Converte o DataFrame para o formato de lista de dicionários que o orquestrador espera
    final_translations_for_cvc = df_translations.to_dict('records')

    print(f"\n✅ Arquivo '{validation_filename}' lido com sucesso!")
    print(f"--- {len(final_translations_for_cvc)} itens carregados para validação ---")
    display(df_translations.head()) # Mostra uma pré-visualização
    print("\n✅ Itens prontos. Execute a Célula 6 para iniciar a análise do comitê.")

except Exception as e:
    print(f"🚨 Erro ao ler ou processar o arquivo. Verifique se o arquivo está correto e use o separador ';'. Erro: {e}")

Por favor, faça o upload do arquivo 'traducoes_finais.csv' que foi salvo pela Célula 3.


Saving traducoes_finaisCISMA.csv to traducoes_finaisCISMA.csv

✅ Arquivo 'traducoes_finaisCISMA.csv' lido com sucesso!
--- 14 itens carregados para validação ---


,item_original,traducao_final
0,¿Cómo me encuentro ahora?,Como estou me sentindo agora?
1,¿Cuánto dolor tengo en este momento?,Quanta dor estou sentindo neste momento?
2,¿Me siento relajado/a?,Eu me sinto relaxado/a?
3,¿Me siento animado/a?,Eu me sinto animado/a?
4,¿Me siento acompañado/a?,Eu me sinto acompanhado?



✅ Itens prontos. Execute a Célula 6 para iniciar a análise do comitê.


In [ ]:
#@title Verificar as configurações para avaliação dos juízes

from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY2')
TEMPERATURA = 0.2
CONTEXTO = """

**Objetivo:** Este instrumento, chamado CISMA, mede o impacto de sessões de musicoterapia no bem-estar de adultos. A tradução deve ser adequada para o português brasileiro.

**--- Manual de Estilo e Diretrizes Metodológicas (Regras) ---**

1.  **Equivalência Semântica:** A tradução deve manter o significado exato do item original em espanhol. Deve-se preservar a temporalidade imediata dos itens (foco no "aqui e agora"), evitando interpretações que generalizem o estado do paciente.

2.  **Estrutura do Item (CRÍTICO):** As perguntas devem ser interrogativas diretas (ex: 'Eu sinto... ?', 'A musicoterapia me ajudou...?', 'As sesssões de musicoterapia...?').

3.  **Perspectiva Narrativa (CRÍTICO):** Este é um instrumento de autorrelato. Os itens devem ser formulados na perspectiva da primeira pessoa ('Eu', 'meu', 'me'). A tradução NÃO PODE mudar para a segunda pessoa ('você', 'seu'). Esta regra é crítica.

4.  **Clareza e Simplicidade:** A linguagem deve ser simples, direta e sem ambiguidades, adequada para pacientes. Cada item deve fazer apenas UMA pergunta. As traduções devem ser concisas, mantendo o espírito de um instrumento "eficiente e breve", sem jargões técnicos.

5.  **Tom:** O tom deve ser neutro e profissional, adequado para um instrumento formal de autorrelato. Evitar gírias ou linguagem excessivamente casual.

6.  **Padrão de Avaliação:** Sua avaliação deve ser extremamente rigorosa. Mesmo desvios menores dessas diretrizes, especialmente em relação à **Estrutura do Item (Regra #2)** e à **Perspectiva Narrativa (Regra #3)**, são considerados falhas significativas.

7. **Proposição de mudanças:** Em casos que julgar necessário proponha nova construção do item com o mesmo conteúdo.
"""

In [ ]:
# Célula 6: ORQUESTRAÇÃO DA ANÁLISE DE JUÍZES E CÁLCULO DO CVC

import numpy as np
import time
import google.generativeai as genai
import json
from google.generativeai.types import GenerationConfig
from google.api_core.exceptions import GoogleAPICallError
from google.colab import userdata


#@title **Rodar Análise do Comitê e Calcular CVC**

#======================================================================
# ETAPA 1: INICIALIZAÇÃO DOS CLIENTES DE API (Mantida)
#======================================================================
print("⚙️ Inicializando clientes de API para a análise dos juízes...")
try:
    generation_config = GenerationConfig(temperature=TEMPERATURA)
    genai.configure(api_key=GEMINI_API_KEY)
    print("✅ Clientes de API prontos.")
except Exception as e:
    raise RuntimeError(f"Falha ao inicializar: {e}")
# (A função de retentativa permanece a mesma)
def call_gemini_with_retry(agent_name, model, prompt, config, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt, generation_config=config); return response
        except Exception as e:
            error_message = str(e).lower()
            if any(code in error_message for code in ['429', '500', '503', 'resource exhausted']):
                wait_time = 20 * (2 ** attempt); print(f"  [AVISO {agent_name}] Erro temporário detectado. Esperando {wait_time}s... ({attempt + 1}/{max_retries})"); time.sleep(wait_time)
            else:
                print(f"  [ERRO GERAL {agent_name}] Erro não recuperável: {e}. Falha imediata."); return None
    print(f"  [ERRO FINAL {agent_name}] Todas as {max_retries} tentativas falharam."); return None

#======================================================================
# ETAPA 2: ORQUESTRAÇÃO DA AVALIAÇÃO
#======================================================================
print("\n🚀 Iniciando a avaliação dos itens pelo comitê...")
try:
    if 'final_translations_for_cvc' not in globals():
        raise NameError("A lista de traduções não foi carregada. Por favor, execute a Célula 5 primeiro.")

    all_judge_scores = []
    for i, item_pair in enumerate(final_translations_for_cvc):
        original = item_pair['item_original']; final = item_pair['traducao_final']
        print(f"--- Avaliando Item {i+1}/{len(final_translations_for_cvc)}: '{final}' ---")
        item_scores = {'clarity': [], 'pertinence': [], 'relevance': [], 'justifications': []}

        for idx, judge in enumerate(committee):
            evaluation = judge.evaluate(original, final, CONTEXTO)

            # --- [MUDANÇA CRUCIAL] Converte a categoria de texto em nota numérica ---
            clarity_score = CATEGORY_TO_SCORE_MAP.get(evaluation.get('clarity_category'), 0)
            pertinence_score = CATEGORY_TO_SCORE_MAP.get(evaluation.get('pertinence_category'), 0)
            relevance_score = CATEGORY_TO_SCORE_MAP.get(evaluation.get('relevance_category'), 0)

            item_scores['clarity'].append(clarity_score)
            item_scores['pertinence'].append(pertinence_score)
            item_scores['relevance'].append(relevance_score)

            justification = f"J{idx+1}: {evaluation.get('brief_justification', 'N/A')}"
            item_scores['justifications'].append(justification)
            time.sleep(1.5)

        all_judge_scores.append({
            'item_final': final, 'clarity_scores': item_scores['clarity'],
            'pertinence_scores': item_scores['pertinence'], 'relevance_scores': item_scores['relevance'],
            'justifications': " | ".join(item_scores['justifications'])
        })
    print("\n✅ Avaliação do comitê concluída.")

    # (A Etapa 3 de cálculo do CVC e relatório não precisa de nenhuma mudança)
    #======================================================================
    # ETAPA 3: CÁLCULO DO CVC E RELATÓRIO FINAL GRANULAR
    #======================================================================
    print("\n📊 Calculando o Coeficiente de Validade de Conteúdo (CVC)...")
    # ... (cole aqui o resto do código da Célula 6, ele funcionará perfeitamente) ...
    def calculate_cvc(scores_df):
        num_judges = len(scores_df.columns); v_max = 5
        mx = scores_df.mean(axis=1); cvci = mx / v_max
        pei = (1 / num_judges) ** num_judges
        cvc_c = cvci - pei; cvc_t = cvci.mean() - pei
        return cvc_c, cvc_t
    clarity_data = pd.DataFrame([item['clarity_scores'] for item in all_judge_scores], columns=[f'Clareza_J{i+1}' for i in range(len(committee))])
    pertinence_data = pd.DataFrame([item['pertinence_scores'] for item in all_judge_scores], columns=[f'Pert_J{i+1}' for i in range(len(committee))])
    relevance_data = pd.DataFrame([item['relevance_scores'] for item in all_judge_scores], columns=[f'Rel_J{i+1}' for i in range(len(committee))])
    cvc_clarity_items, cvc_clarity_total = calculate_cvc(clarity_data)
    cvc_pertinence_items, cvc_pertinence_total = calculate_cvc(pertinence_data)
    cvc_relevance_items, cvc_relevance_total = calculate_cvc(relevance_data)
    report_df = pd.DataFrame({'Item Traduzido': [item['item_final'] for item in all_judge_scores]})
    report_df = pd.concat([report_df, clarity_data, pertinence_data, relevance_data], axis=1)
    report_df['CVC Clareza'] = cvc_clarity_items; report_df['CVC Pertinência'] = cvc_pertinence_items; report_df['CVC Relevância'] = cvc_relevance_items
    report_df['Justificativas'] = [item['justifications'] for item in all_judge_scores]
    def highlight_low_cvc(val):
        if isinstance(val, (int, float)): return 'background-color: #FFDDDD' if val < 0.80 else ''
        return ''
    numeric_cols = [col for col in report_df.columns if 'CVC' in col]
    styled_report = report_df.style.applymap(highlight_low_cvc, subset=numeric_cols).format({col: '{:.3f}' for col in numeric_cols})
    print("\n--- RELATÓRIO FINAL DE VALIDADE DE CONTEÚDO ---")
    print(f"Ponto de corte recomendado (Hernández-Nieto, 2002): CVC > 0.80")
    print("-" * 50); print(f"CVC Total (Clareza): {cvc_clarity_total:.3f}"); print(f"CVC Total (Pertinência): {cvc_pertinence_total:.3f}"); print(f"CVC Total (Relevância): {cvc_relevance_total:.3f}")
    print("-" * 50); print("(Itens com CVC abaixo do ponto de corte estão destacados em vermelho)")
    display(styled_report)
    print("\n💾 Exportando o relatório detalhado para um arquivo Excel...")
    excel_filename = "relatorio_validacao_cvc.xlsx"
    styled_report.to_excel(excel_filename, engine="openpyxl", index=False)
    files.download(excel_filename)
    print(f"✅ Arquivo '{excel_filename}' gerado e download iniciado!")
except Exception as e:
    print(f"🚨 UM ERRO INESPERADO OCORREU: {e}")

⚙️ Inicializando clientes de API para a análise dos juízes...
✅ Clientes de API prontos.

🚀 Iniciando a avaliação dos itens pelo comitê...
--- Avaliando Item 1/14: 'Como estou me sentindo agora?' ---
--- Avaliando Item 2/14: 'Quanta dor estou sentindo neste momento?' ---
--- Avaliando Item 3/14: 'Eu me sinto relaxado/a?' ---
--- Avaliando Item 4/14: 'Eu me sinto animado/a?' ---
--- Avaliando Item 5/14: 'Eu me sinto acompanhado?' ---
--- Avaliando Item 6/14: 'A sessão de musicoterapia me ajudou a me expressar?' ---
--- Avaliando Item 7/14: 'A sessão de musicoterapia me ajudou a me divertir?' ---
--- Avaliando Item 8/14: 'A sessão de musicoterapia me ajudou a lembrar ou a ativar minha memória?' ---
--- Avaliando Item 9/14: 'A sessão de musicoterapia me distraiu de minhas preocupações?' ---
--- Avaliando Item 10/14: 'A sessão de musicoterapia me ajudou a me conectar com meus sentimentos?' ---
--- Avaliando Item 11/14: 'A sessão de musicoterapia me ajudou a me relacionar com outras pessoas

/tmp/ipython-input-2484031214.py:100: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_report = report_df.style.applymap(highlight_low_cvc, subset=numeric_cols).format({col: '{:.3f}' for col in numeric_cols})


,Item Traduzido,Clareza_J1,Clareza_J2,Clareza_J3,Clareza_J4,Clareza_J5,Pert_J1,Pert_J2,Pert_J3,Pert_J4,Pert_J5,Rel_J1,Rel_J2,Rel_J3,Rel_J4,Rel_J5,CVC Clareza,CVC Pertinência,CVC Relevância,Justificativas
0,Como estou me sentindo agora?,5,5,4,5,3,5,5,5,5,4,5,5,5,5,4,0.880,0.960,0.960,"J1: The translated item perfectly adheres to all rules, including Item Structure (Rule #2) and Narrative Perspective (Rule #3), maintaining excellent clarity and semantic equivalence. | J2: The translation is excellent. It perfectly adheres to the critical rules, using a direct interrogative structure (Rule #2) and the first-person perspective (Rule #3). The phrasing is natural and unambiguous in Brazilian Portuguese. | J3: The translation accurately reflects the original item's meaning and temporal focus (Rule #1). However, the item structure is problematic. While understandable, 'Como estou me sentindo agora?' is a statement disguised as a question, violating Rule #2. It lacks a clear interrogative form. A direct question is required. | J4: The translation is excellent. It perfectly adheres to all rules, especially the critical ones. It maintains the direct interrogative structure (Rule #2) and the first-person perspective (Rule #3) with impeccable clarity (Rule #4). | J5: The item is clear and directly asks about the respondent's current feelings, aligning with Rule #4 (Clarity and Simplicity) and Rule #2 (Item Structure - interrogative direct). However, it uses the second person ('me sentindo'), violating Rule #3 (Perspectiva Narrativa)."
1,Quanta dor estou sentindo neste momento?,5,5,4,5,3,5,5,5,5,4,5,5,5,5,4,0.880,0.960,0.960,"J1: The item perfectly adheres to all rules, maintaining semantic equivalence, correct interrogative structure (Rule #2), and first-person perspective (Rule #3). | J2: The item is perfectly clear and adheres to all rules, including the critical Item Structure (Rule #2) and Narrative Perspective (Rule #3). The phrasing is natural for Brazilian Portuguese. | J3: The translation accurately reflects the original item's meaning and temporal focus (Rule #1). It maintains a clear and direct question about current pain levels. However, the phrasing 'Quanta dor estou sentindo' is slightly awkward and could be improved for natural flow, though it doesn't violate any rules. | J4: A tradução está perfeitamente alinhada com todas as regras, mantendo a perspectiva em primeira pessoa (Regra #3) e a estrutura interrogativa direta (Regra #2), com clareza impecável. | J5: The item is clear and directly asks about current pain levels, aligning with the goal of measuring the immediate impact of music therapy. It uses first-person perspective ('estou sentindo'), adhering to Rule #3."
2,Eu me sinto relaxado/a?,5,5,5,5,4,5,5,5,5,4,5,5,5,5,4,0.960,0.960,0.960,"J1: The translated item perfectly adheres to all rules, maintaining semantic equivalence, correct interrogative structure (Rule #2), first-person perspective (Rule #3), and clarity (Rule #4). | J2: The translation is a perfect rendering, fully compliant with all rules, including the critical Item Structure (Rule #2) and Narrative Perspective (Rule #3). | J3: The translation accurately reflects the original item's meaning and maintains the immediate temporal focus (Rule #1). It adheres to the required interrogative structure (Rule #2) and first-person perspective (Rule #3). The language is simple and direct (Rule #4), and the tone is neutral (Rule #5). | J4: The translation is a perfect equivalent, adhering strictly to all rules, including the critical Item Structure (Rule #2) and Narrative Perspective (Rule #3). | J5: The item is interrogative and uses first-person perspective ('Eu me sinto'), adhering to Rule #2 and Rule #3. It's clear and directly assesses a key outcome – relaxation – relevant to music therapy's impact on well-being."
3,Eu me sinto animado/a?,5,5,5,5,3,5,5,5,5,4,5,5,5,5,4,0.920,0.960,0.960,"J1: The translated item perfectly adheres to all rules, especially Rule #2 (It


💾 Exportando o relatório detalhado para um arquivo Excel...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Arquivo 'relatorio_validacao_cvc.xlsx' gerado e download iniciado!


Lista de modelos:

Modelos disponíveis na Google Generative AI API:

- models/gemini-2.5-pro-preview-03-25
- models/gemini-2.5-flash-preview-05-20
- models/gemini-2.5-flash
- models/gemini-2.5-flash-lite-preview-06-17
- models/gemini-2.5-pro-preview-05-06
- models/gemini-2.5-pro-preview-06-05
- models/gemini-2.5-pro
- models/gemini-2.0-flash-exp
- models/gemini-2.0-flash
- models/gemini-2.0-flash-001
- models/gemini-2.0-flash-exp-image-generation
- models/gemini-2.0-flash-lite-001
- models/gemini-2.0-flash-lite
- models/gemini-2.0-flash-preview-image-generation
- models/gemini-2.0-flash-lite-preview-02-05
- models/gemini-2.0-flash-lite-preview
- models/gemini-2.0-pro-exp
- models/gemini-2.0-pro-exp-02-05
- models/gemini-exp-1206
- models/gemini-2.0-flash-thinking-exp-01-21
- models/gemini-2.0-flash-thinking-exp
- models/gemini-2.0-flash-thinking-exp-1219
- models/gemini-2.5-flash-preview-tts
- models/gemini-2.5-pro-preview-tts
- models/learnlm-2.0-flash-experimental
- models/gemma-3-1b-it
- models/gemma-3-4b-it
- models/gemma-3-12b-it
- models/gemma-3-27b-it
- models/gemma-3n-e4b-it
- models/gemma-3n-e2b-it
- models/gemini-flash-latest
- models/gemini-flash-lite-latest
- models/gemini-pro-latest
- models/gemini-2.5-flash-lite
- models/gemini-2.5-flash-image-preview
- models/gemini-2.5-flash-preview-09-2025
- models/gemini-2.5-flash-lite-preview-09-2025
- models/gemini-robotics-er-1.5-preview

In [ ]:
 excel_filename = "relatorio_validacao_cvc.xlsx"

    # Usa o método .to_excel() do objeto Styler para salvar
    # 'engine="openpyxl"' especifica qual biblioteca usar
    # 'index=False' remove a coluna de índice do Pandas (0, 1, 2...)
styled_report.to_excel(excel_filename, engine="openpyxl", index=False)

    # Aciona o download do arquivo gerado
files.download(excel_filename)

print(f"✅ Arquivo '{excel_filename}' gerado e download iniciado!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Arquivo 'relatorio_validacao_cvc.xlsx' gerado e download iniciado!
